In [ ]:
#| default_exp InsuranceAI
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#| hide
import pandas as pd
from InsuranceAI.utils import read_data, select_variables


# InsuranceAI

> Testing for Asymmetric Information in Insurance with Deep Learning

We read the data and we convert the variables to their correct types:

In [ ]:
data88 = read_data("j88")

## selecting  variables
We will work with a subset of the variables. Our endogeneous variables will be `Comprehensive coverage` for the choice of contract $y_1$ and `Not at fault` for the occurrence of a claim at fault $y_2$.
We also use the variable `Duration` to weight the observations, and 9 categorical covariates.

In [ ]:
y1, y2, w, X = select_variables(data88)

In [ ]:
X.describe()

,Group,Male,Occupation,Region,Renault,Age category car,Age category insuree,Car use,Zone
count,6333,6333,6333,6333,6333,6333,6333,6333,6333
unique,6,2,8,10,2,12,9,4,5
top,3,1,2,5,0,11,1,2,6
freq,1272,3971,2865,1019,5046,1100,1354,2876,2626


In [ ]:
lhs_vars = pd.DataFrame({
    'Comprehensive coverage': y1, 
    'Claim at fault': y2, 'Duration': w
})
lhs_vars.describe()


,Comprehensive coverage,Claim at fault,Duration
count,6333.000000,6333.000000,6333.000000
mean,0.368704,0.068530,0.630466
std,0.482491,0.252673,0.338334
min,0.000000,0.000000,0.002740
25%,0.000000,0.000000,0.304110
50%,0.000000,0.000000,0.657534
75%,1.000000,0.000000,0.997260
max,1.000000,1.000000,0.997260


The matrix of $(y_1,y_2)$ is as follows:

In [ ]:
pd.crosstab(lhs_vars['Comprehensive coverage'], lhs_vars['Claim at fault'])

Claim at fault,0,1
Comprehensive coverage,,
0,3696,302
1,2203,132


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()